# Compare Result

this notebook created for comparing result between predicted result from ML model and truth set 

In [1]:
!pip install venn

# BAT

## Result V2

In [1]:
import csv
from venn import venn
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

from functools import reduce

In [2]:
columns = ["ID_Query","ID_Target","sequence_identity","alignment_length","number_of_mismatches","number_of_gap_openings", "start-position_in_query", "end-position_in_query", "start-position_in_target","end-position_in_target","E-value","bit_score"]

In [3]:
def plot_ven_AMP_NonAMP(sample_test,sample_groundtruth,prediced_set,truth_set):
    Samples = {
        sample_test: set(prediced_set["ID"]),
        sample_groundtruth: set(truth_set['ID_Query']),
        }
    venn(Samples)
    # venn(Samples, fmt="{percentage:.2f}%")    
    
def accuracy_cal( predict , total_AMPs, per):
    accuracy = (predict/total_AMPs )*100
    print(per+" Accuracy Percentage = %.2f" % accuracy)
    return accuracy

# ---------------------------------------

From A)  first we use prediction result ( which contains both NonAMP  and AMP  )  ( Purple bubble here )  intersect with a ground truth set.

Next , In B) we exclude the non AMPs out from prediction results , we see a high number in ( purple bubble). This means  Most of the sequence predict to be AMP rather than to be a non AMP (you  can see the  number  in purple part between A and B) which is not likely to be  true . So in this case show a Very high false positive rate on the prediction (i.e., predict Non AMP to be AMP , high amount in purple color in Venn diagram),  
(If we calculate non amp  625K-579K = 46K.)

Next if we look into yellow bubble , there is a remaining  number  in yellow which we consider it as a false negative  because the model predict these  as the Non AMPs.

In [66]:
for key, value in method_dict.items():
    method = value
    print("-----------------"+ method +"-------------------------")
    Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_2_"+method+"/Bat.len10.MLResult.plk")
    Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                      ,encoding = "ISO-8859-1", sep='\t',header=0)
    
    Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
    Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]
    
    # AMP and Non AMP compare with ground truth
    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                Bat_ML,
    #                Bat_len10_truth
    #               )
    # AMP only compares with ground truth
    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                Bat_AMPs_ML,
    #                Bat_len10_truth
    #               )
     
    # AMP only compares with filtering 80% seq identity on ground truth
    filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
    filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
    AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
    accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 
    
    # AMP only compares with filtering 100% seq identity on ground truth
    filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
    filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                    Bat_AMPs_ML,
    #                    filtered_BAT_len10_truth
    #               )
    
    
    # AMP match Truthset
    AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
    accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")
    
    AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
    # 100%
    accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")
    
    print("Total result " + str(len(Bat_ML)) )
    print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
    print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
    print("Total match AMP " + str(len(AMP_match_Truth )))
    print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
    print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
    print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )
    
    
    _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
    _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
    _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
    _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
    _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
    _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
    _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
    _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)
    
    #print("-------------------------------------------------")
    

-----------------Ensemble-------------------------
80% Accuracy Percentage = 70.81
no condi Accuracy Percentage = 54.92
100% Accuracy Percentage = 73.76
Total result 422796
Total predicted AMP 57335
Total predicted NonAMP 365461
Total match AMP 3992
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1206
Total AMP with_100 from MMSEQ: 1635
-----------------FastAI-------------------------
80% Accuracy Percentage = 77.14
no condi Accuracy Percentage = 64.75
100% Accuracy Percentage = 77.61
Total result 422796
Total predicted AMP 93036
Total predicted NonAMP 329760
Total match AMP 4707
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1269
Total AMP with_100 from MMSEQ: 1635
-----------------RF-------------------------
80% Accuracy Percentage = 72.54
no condi Accuracy Percentage = 56.21
100% Accuracy Percentage = 74.13
Total result 422796
Total predicted AMP 82525
Total predicted NonAMP 340271
Total match AMP 4086
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1212
Total AMP wi

In [67]:
_df_to_report

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,57335,365461,3992,54.92,1206,73.76,3546.0,70.81
FastAI,93036,329760,4707,64.75,1269,77.61,3863.0,77.14
RF,82525,340271,4086,56.21,1212,74.13,3633.0,72.54
XGBoost,98533,324263,4718,64.91,1292,79.02,4018.0,80.23
SVM,94764,328032,4912,67.57,1350,82.57,4210.0,84.07
AmPep30,137580,285216,1485,20.43,195,11.93,772.0,15.42
CNN_RMSpop,57487,365309,2776,38.19,803,49.11,2436.0,48.64
CNN_Adam,114965,307831,4784,65.81,1323,80.92,4042.0,80.71
ProtCNN,97179,325617,4454,61.27,1305,79.82,3852.0,76.92
DNN,117656,305140,2650,36.46,847,51.80,1907.0,38.08


In [ ]:
# <= 200

In [68]:
for key, value in method_dict.items():
    method = value
    print("-----------------"+ method +"-------------------------")
    Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_2_"+method+"/Bat.len10.MLResult.plk")
    Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output200/bat.len200.tsv"
                                      ,encoding = "ISO-8859-1", sep='\t',header=0)
    Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
    Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
    Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]
    
    # AMP and Non AMP compare with ground truth
    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                Bat_ML,
    #                Bat_len10_truth
    #               )
    # AMP only compares with ground truth
    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                Bat_AMPs_ML,
    #                Bat_len10_truth
    #               )
 
    # AMP only compares with filtering 80% seq identity on ground truth
    filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
    filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
    AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
    accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%")
    
    # AMP only compares with filtering 100% seq identity on ground truth
    filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
    filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

    #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
    #                    Bat_AMPs_ML,
    #                    filtered_BAT_len10_truth
    #               )
    
    
    # AMP match Truthset
    AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
    accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")
    
    AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
    # 100%
    accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")
    
    print("Total result " + str(len(Bat_ML)) )
    #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
    #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
    #print("Total match AMP " + str(len(AMP_match_Truth )))
    print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
    #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
    #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )
    
    
    _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
    _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
    _df_to_report_200.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
    _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
    _df_to_report_200.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
    _df_to_report_200.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
    _df_to_report_200.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
    _df_to_report_200.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)
    
    #print("-------------------------------------------------")

-----------------Ensemble-------------------------
80% Accuracy Percentage = 73.36
no condi Accuracy Percentage = 62.18
100% Accuracy Percentage = 70.91
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------FastAI-------------------------
80% Accuracy Percentage = 79.59
no condi Accuracy Percentage = 70.14
100% Accuracy Percentage = 75.44
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------RF-------------------------
80% Accuracy Percentage = 74.15
no condi Accuracy Percentage = 63.39
100% Accuracy Percentage = 71.27
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------XG-------------------------
80% Accuracy Percentage = 80.81
no condi Accuracy Percentage = 70.57
100% Accuracy Percentage = 76.79
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------SVM-------------------------
80% Accuracy Percentage = 83.92
no condi Accuracy Percentage = 73.24
100% Accuracy Percentage = 80.54
Total result 339238
Total AMP  from MMSEQ: 5613
--------------

In [69]:
_df_to_report_200

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,52239,286999,3490,62.18,1002,70.91,3184.0,73.36
FastAI,77416,261822,3937,70.14,1066,75.44,3454.0,79.59
RF,75547,263691,3558,63.39,1007,71.27,3218.0,74.15
XGBoost,85139,254099,3961,70.57,1085,76.79,3507.0,80.81
SVM,84214,255024,4111,73.24,1138,80.54,3642.0,83.92
AmPep30,107291,231947,908,16.18,187,13.23,675.0,15.55
CNN_RMSpop,53220,286018,2356,41.97,637,45.08,2131.0,49.10
CNN_Adam,102237,237001,4081,72.71,1115,78.91,3576.0,82.40
ProtCNN,87116,252122,3830,68.23,1098,77.71,3441.0,79.29
DNN,109315,229923,2077,37.00,740,52.37,1604.0,36.96


In [ ]:
# Transpi

# Result V3 

##  Full

In [129]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_3/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)

        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        print("Total result " + str(len(Bat_ML)) )
        print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 80.39
no condi Accuracy Percentage = 67.11
100% Accuracy Percentage = 78.04
Total result 422796
Total predicted AMP 102877
Total predicted NonAMP 319919
Total match AMP 4878
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1276
Total AMP with_100 from MMSEQ: 1635
-----------------RF-------------------------
80% Accuracy Percentage = 83.47
no condi Accuracy Percentage = 66.12
100% Accuracy Percentage = 80.86
Total result 422796
Total predicted AMP 103961
Total predicted NonAMP 318835
Total match AMP 4806
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1322
Total AMP with_100 from MMSEQ: 1635
-----------------XG-------------------------
80% Accuracy Percentage = 86.28
no condi Accuracy Percentage = 72.46
100% Accuracy Percentage = 83.49
Total result 422796
Total predicted AMP 128782
Total predicted NonAMP 294014
Total match AMP 5267
Total AMP  from MMSEQ:

In [130]:
_df_to_report

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,102877,319919,4878,67.11,1276,78.04,4026,80.39
RF,103961,318835,4806,66.12,1322,80.86,4180,83.47
XGBoost,128782,294014,5267,72.46,1365,83.49,4321,86.28
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,109648,313148,5161,71.00,1382,84.53,4283,85.52
CNN_Adam,127115,295681,5264,72.42,1344,82.20,4300,85.86
ProtCNN,112936,309860,5021,69.07,1325,81.04,4138,82.63
DNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Only <= 200 aa

In [131]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_3/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output200/bat.len200.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%")

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report_200.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report_200.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report_200.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report_200.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 81.20
no condi Accuracy Percentage = 72.37
100% Accuracy Percentage = 75.87
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------RF-------------------------
80% Accuracy Percentage = 83.41
no condi Accuracy Percentage = 71.07
100% Accuracy Percentage = 78.91
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------XG-------------------------
80% Accuracy Percentage = 85.44
no condi Accuracy Percentage = 75.40
100% Accuracy Percentage = 81.95
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
80% Accuracy Percentage = 84.98
no condi Accuracy Percentage = 73.79
100% Accuracy Percentage = 83.30
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------CNN_Adam-------------------------
80% Accur

In [132]:
_df_to_report_200

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,85356,253882,4062,72.37,1072,75.87,3524,81.20
RF,98293,240945,3989,71.07,1115,78.91,3620,83.41
XGBoost,115949,223289,4232,75.40,1158,81.95,3708,85.44
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,98639,240599,4142,73.79,1177,83.30,3688,84.98
CNN_Adam,114486,224752,4185,74.56,1137,80.47,3697,85.18
ProtCNN,102765,236473,4066,72.44,1124,79.55,3565,82.14
DNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Result V3 Recalibrate

In [144]:
Bat_AMPs_ML

,ID,length,class
7,SOAP.k25.C306082.p1,110,0
13,SOAP.k25.C307230.p1,107,0
18,SOAP.k25.C308218.p1,116,0
19,SOAP.k25.C308334.p1,118,0
35,SOAP.k25.C310344.p1,112,0
...,...,...,...
984,Velvet.k61.NODE_38208_length_552_cov_61.673912.p1,194,0
989,Velvet.k61.NODE_51327_length_365_cov_13.287671.p1,141,0
995,Velvet.k61.NODE_6475_length_400_cov_9.495000.p1,146,0
996,Velvet.k61.NODE_6568_length_308_cov_32.344154.p1,112,0


In [145]:
Bat_len10_truth.prot_id

0       TRINITY_DN5330_c0_g1_i1.p4
1       TRINITY_DN5330_c0_g1_i1.p1
2      TRINITY_DN18336_c0_g1_i2.p1
3      TRINITY_DN18336_c0_g1_i2.p3
4      TRINITY_DN24680_c0_g1_i1.p1
                  ...             
245        SOAP.k61.scaffold787.p2
246        SOAP.k61.scaffold787.p1
247             SOAP.k61.C37063.p2
248             SOAP.k61.C38569.p2
249             SOAP.k61.C41337.p1
Name: prot_id, Length: 250, dtype: object

# Result V3 Transpi

## Test False positive

In [183]:

from Bio import SeqIO
Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_3/result_CNN_Adam/Bat.len10.MLResult.plk")
Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.csv"
                                          , sep=',',header=0)
Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]
AMP_FP = Bat_AMPs_ML[~Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))]
AMP_FP

,ID,length,class
4,SOAP.k25.C304386.p1,104,0
7,SOAP.k25.C306082.p1,110,0
13,SOAP.k25.C307230.p1,107,0
14,SOAP.k25.C307380.p1,114,0
16,SOAP.k25.C308074.p1,116,0
...,...,...,...
992,Velvet.k61.NODE_58895_length_568_cov_53.484154.p1,110,0
995,Velvet.k61.NODE_6475_length_400_cov_9.495000.p1,146,0
996,Velvet.k61.NODE_6568_length_308_cov_32.344154.p1,112,0
997,Velvet.k61.NODE_7143_length_732_cov_25.393442.p1,102,0


In [177]:
fastas = "/mnt/vdb/Bat/transpi/SRR12103592.combined.okay.fa.transdecoder.pep"
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
_BATpep_df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
_BATpep_df  

,ID,Sequence,length
0,SOAP.k25.C303246.p1,QKVLQAAGPSTTTETETIAKYEIMDGAPVKGESIPIRLFLAGYDPT...,101
1,SOAP.k25.C303708.p1,PITWGRKWNIENGCARTHSQDDYSPGSQAQGESGTASHPRRGHLEM...,102
2,SOAP.k25.C304032.p1,LRNHSPLMSFGASFVSFLNAMMTFEEEKMQLACDDLRTTEKLCESE...,103
3,SOAP.k25.C304284.p1,ESTDQISPYGNSTVTQPSDSGWQYNETHTSLKQNTPRNTSKLYIGL...,104
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,104
...,...,...,...
11000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,MSPSQAVYIVPSKGRLIGGLRDTPSYEHFQEDFSTCSLCTFRDLCA...,106
11001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,QDLENAATGDAAVHQRIASLPVEVQEVSLLDKITDKESGERLSKMV...,105
11002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,ARAALAMPVKGGTKCIKYLLLGFNFVFWLAGIAVLAIGLWLRFDSQ...,233
11003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,MSCKPQCSLNHLPTPCARQSAPFRIPAEFLYLVLLLVEGAPFSNFS...,367


In [178]:
AMP_FP = _BATpep_df[_BATpep_df["ID"].isin(AMP_FP.ID.to_list())]
AMP_FP

,ID,Sequence,length
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,104
7,SOAP.k25.C306082.p1,IQYRHQEQKDGCYLTQFKQFSRAGRASIILIINRIRERLQRMKPIK...,110
13,SOAP.k25.C307230.p1,AAGVMAGQAFRKFLPLFDRVLVERSAAETVTKGGIMLPEKAQGKVL...,107
14,SOAP.k25.C307380.p1,WKEERRAWKEAGKASIARPSGDTIIWLLPVLHSPTGSYLLPPSTTT...,114
16,SOAP.k25.C308074.p1,FNPLEVSVLRYPREPSSAPHQASVCRLLQGAGSQPNVTACQPGHPS...,116
...,...,...,...
10992,Velvet.k61.NODE_58895_length_568_cov_53.484154.p1,KHVAALLELLPSHTQSNLVRCWRRASCTTLSSAPPNTPREPAPVAL...,110
10995,Velvet.k61.NODE_6475_length_400_cov_9.495000.p1,LTGRAAPPVGPLADWGPVSLQLVELKNGETYNGHLVSCDNWMNINL...,146
10996,Velvet.k61.NODE_6568_length_308_cov_32.344154.p1,SEGERKNLKSQALGNRGITRVSISLHLFILALKRHFPVSNLCHLVG...,112
10997,Velvet.k61.NODE_7143_length_732_cov_25.393442.p1,WRLGLSSCHSTVPPQTSYTSWIPFNLKGERAGKSIKTPIKQVSQLN...,102


In [179]:
with open("/mnt/vdb/thesis/amp_final/ampV3/AMPs.FP.fasta", 'w') as file:
    for index, row in AMP_FP.iterrows():
        header = row['ID']
        seq = row['Sequence']
        # print(header)
        file.write('>'+ header + '\n')
        file.write(seq + '\n')
           

In [ ]:
#-------------------------- ---------------------------#

In [169]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "DNN" : "DNN"
}


_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

In [174]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_3/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.csv"
                                          , sep=',',header=0)
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(AMP_match_Truth ))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 18.54
Total result 11005
Total match AMP 71
Total AMP from Tanspi: 383
-----------------RF-------------------------
no condi Accuracy Percentage = 13.05
Total result 11005
Total match AMP 50
Total AMP from Tanspi: 383
-----------------XG-------------------------
no condi Accuracy Percentage = 20.63
Total result 11005
Total match AMP 79
Total AMP from Tanspi: 383
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
no condi Accuracy Percentage = 14.6

In [171]:
_df_to_report

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,2098,8907,71,18.54
RF,1538,9467,50,13.05
XGBoost,2236,8769,79,20.63
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,1663,9342,56,14.62
CNN_Adam,1823,9182,69,18.02
ProtCNN,1712,9293,55,14.36
DNN,NaN,NaN,NaN,NaN


## Only <= 200 aa

In [149]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_3/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.csv"
                                          , sep=',',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth.prot_id)
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 13.32
Total result 6872
Total AMP from Tanspi: 383
-----------------RF-------------------------
no condi Accuracy Percentage = 8.62
Total result 6872
Total AMP from Tanspi: 383
-----------------XG-------------------------
no condi Accuracy Percentage = 15.40
Total result 6872
Total AMP from Tanspi: 383
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
no condi Accuracy Percentage = 9.40
Total result 6872
Total AMP from Tanspi: 383
---------------

In [150]:
_df_to_report_200

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,1470,5402,51,13.32
RF,1197,5675,33,8.62
XGBoost,1739,5133,59,15.40
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,1297,5575,36,9.40
CNN_Adam,1442,5430,50,13.05
ProtCNN,1346,5526,41,10.70
DNN,NaN,NaN,NaN,NaN


# Result V3  model recal_nr

In [305]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "DNN" : "DNN"
}


_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

## CD100

In [306]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_3_recal_nr/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)

        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        print("Total result " + str(len(Bat_ML)) )
        print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 84.60
no condi Accuracy Percentage = 70.68
100% Accuracy Percentage = 81.71
Total result 422796
Total predicted AMP 102458
Total predicted NonAMP 320338
Total match AMP 5138
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1336
Total AMP with_100 from MMSEQ: 1635
-----------------RF-------------------------
80% Accuracy Percentage = 82.67
no condi Accuracy Percentage = 65.39
100% Accuracy Percentage = 80.92
Total result 422796
Total predicted AMP 97966
Total predicted NonAMP 324830
Total match AMP 4753
Total AMP  from MMSEQ: 7269
Total match AMP with_100 1323
Total AMP with_100 from MMSEQ: 1635
-----------------XG-------------------------
80% Accuracy Percentage = 85.06
no condi Accuracy Percentage = 71.08
100% Accuracy Percentage = 83.06
Total result 422796
Total predicted AMP 127651
Total predicted NonAMP 295145
Total match AMP 5167
Total AMP  from MMSEQ: 

In [307]:
_df_to_report

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,102458,320338,5138,70.68,1336,81.71,4237,84.60
RF,97966,324830,4753,65.39,1323,80.92,4140,82.67
XGBoost,127651,295145,5167,71.08,1358,83.06,4260,85.06
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,111109,311687,4982,68.54,1348,82.45,4281,85.48
CNN_Adam,123330,299466,5184,71.32,1331,81.41,4115,82.17
ProtCNN,81138,341658,5094,70.08,1351,82.63,4244,84.74
DNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_3_recal_nr/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output200/bat.len200.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%")

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report_200.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report_200.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report_200.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report_200.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 83.80
no condi Accuracy Percentage = 74.38
100% Accuracy Percentage = 79.55
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------RF-------------------------
80% Accuracy Percentage = 82.37
no condi Accuracy Percentage = 69.98
100% Accuracy Percentage = 78.98
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------XG-------------------------
80% Accuracy Percentage = 84.24
no condi Accuracy Percentage = 73.21
100% Accuracy Percentage = 81.53
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
80% Accuracy Percentage = 85.35
no condi Accuracy Percentage = 73.17
100% Accuracy Percentage = 80.75
Total result 339238
Total AMP  from MMSEQ: 5613
-----------------CNN_Adam-------------------------
80% Accur

In [309]:
_df_to_report_200

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,85845,253393,4175,74.38,1124,79.55,3637,83.80
RF,91363,247875,3928,69.98,1116,78.98,3575,82.37
XGBoost,113393,225845,4109,73.21,1152,81.53,3656,84.24
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,99120,240118,4107,73.17,1141,80.75,3704,85.35
CNN_Adam,108684,230554,4059,72.31,1126,79.69,3490,80.41
ProtCNN,71674,267564,4132,73.61,1157,81.88,3678,84.75
DNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transpi  

In [335]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "DNN" : "DNN"
}
_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])
_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])
_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","DNN"])

In [338]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_3_recal_nr/result_"+method+"/Bat.len10.MLResult.plk")
           # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.csv"
                                          , sep=',',header=0)
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(AMP_match_Truth ))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 16.97
Total result 11005
Total match AMP 65
Total AMP from Tanspi: 383
-----------------RF-------------------------
no condi Accuracy Percentage = 9.40
Total result 11005
Total match AMP 36
Total AMP from Tanspi: 383
-----------------XG-------------------------
no condi Accuracy Percentage = 13.84
Total result 11005
Total match AMP 53
Total AMP from Tanspi: 383
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
no condi 

In [339]:
_df_to_report

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,2093,8912,65,16.97
RF,944,10061,36,9.40
XGBoost,1489,9516,53,13.84
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,1310,9695,40,10.44
CNN_Adam,1565,9440,57,14.88
ProtCNN,1322,9683,48,12.53
DNN,NaN,NaN,NaN,NaN


In [334]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_3_recal_nr/result_"+method+"/Bat.len10.MLResult.plk")
        # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.stringent.csv"
                                          , sep=',',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 10.80
Total result 6872
Total AMP from Tanspi: 250
-----------------RF-------------------------
no condi Accuracy Percentage = 5.20
Total result 6872
Total AMP from Tanspi: 250
-----------------XG-------------------------
no condi Accuracy Percentage = 10.00
Total result 6872
Total AMP from Tanspi: 250
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_3_recal_nr/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
no condi Accuracy Percentage = 6.80
Total result 6872
Total AMP from 

In [333]:
_df_to_report_200

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,1481,5391,47,12.27
RF,756,6116,22,5.74
XGBoost,1120,5752,38,9.92
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,966,5906,27,7.05
CNN_Adam,1156,5716,38,9.92
ProtCNN,1021,5851,39,10.18
DNN,NaN,NaN,NaN,NaN


## V4

### CD100

In [376]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}


_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_4/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)

        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

       # print("Total result " + str(len(Bat_ML)) )
       # print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
       # print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
       # print("Total match AMP " + str(len(AMP_match_Truth )))
       # print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
       # print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue
_df_to_report

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 80.39
no condi Accuracy Percentage = 67.11
100% Accuracy Percentage = 78.04
-----------------RF-------------------------
80% Accuracy Percentage = 84.48
no condi Accuracy Percentage = 67.71
100% Accuracy Percentage = 82.14
-----------------XG-------------------------
80% Accuracy Percentage = 86.20
no condi Accuracy Percentage = 72.55
100% Accuracy Percentage = 84.04
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
-----------------CNN_Adam-------------------------
80% Accuracy Percentage = 83.75
no condi Accuracy Percentage = 70.31
100% Accuracy Percentage = 81.47
-----------------ProtCNN-------------------------
80% Accuracy Percentage = 87.98
no condi Accuracy Percentage = 75.73
100% Accuracy Percentage = 84.77
-----------------BILSTM-------------------------


,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,102877,319919,4878,67.11,1276,78.04,4026,80.39
RF,107694,315102,4922,67.71,1343,82.14,4231,84.48
XGBoost,132220,290576,5274,72.55,1374,84.04,4317,86.20
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_Adam,110795,312001,5111,70.31,1332,81.47,4194,83.75
ProtCNN,160338,262458,5505,75.73,1386,84.77,4406,87.98
BILSTM,123399,299397,5277,72.60,1359,83.12,4238,84.62


In [377]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_4/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output200/bat.len200.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%")

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        #print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        #print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report_200.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report_200.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report_200.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report_200.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue
_df_to_report_200

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 81.20
no condi Accuracy Percentage = 72.37
100% Accuracy Percentage = 75.87
-----------------RF-------------------------
80% Accuracy Percentage = 84.40
no condi Accuracy Percentage = 72.12
100% Accuracy Percentage = 80.40
-----------------XG-------------------------
80% Accuracy Percentage = 85.55
no condi Accuracy Percentage = 75.57
100% Accuracy Percentage = 82.59
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
-----------------CNN_Adam-------------------------
80% Accuracy Percentage = 82.30
no condi Accuracy Percentage = 72.17
100% Accuracy Percentage = 79.69
-----------------ProtCNN-------------------------
80% Accuracy Percentage = 87.00
no condi Accuracy Percentage = 77.61
100% Accuracy Percentage = 83.51
-----------------BILSTM-------------------------


,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,85356,253882,4062,72.37,1072,75.87,3524,81.20
RF,100061,239177,4048,72.12,1136,80.40,3663,84.40
XGBoost,117465,221773,4242,75.57,1167,82.59,3713,85.55
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_Adam,98559,240679,4051,72.17,1126,79.69,3572,82.30
ProtCNN,140962,198276,4356,77.61,1180,83.51,3776,87.00
BILSTM,107339,231899,4210,75.00,1154,81.67,3644,83.96


### Transpi

In [374]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}
_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_4/result_"+method+"/Bat.len10.MLResult.plk")
           # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.stringent.csv"
                                          , sep=',',header=0)
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(AMP_match_Truth ))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue
_df_to_report

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 16.80
Total result 11005
Total match AMP 42
Total AMP from Tanspi: 250
-----------------RF-------------------------
no condi Accuracy Percentage = 7.60
Total result 11005
Total match AMP 19
Total AMP from Tanspi: 250
-----------------XG-------------------------
no condi Accuracy Percentage = 13.20
Total result 11005
Total match AMP 33
Total AMP from Tanspi: 250
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
[Errno 2] No such file or directory:

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,2098,8907,42,16.8
RF,971,10034,19,7.6
XGBoost,1696,9309,33,13.2
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN
CNN_Adam,1377,9628,25,10.0
ProtCNN,2126,8879,46,18.4
BILSTM,1670,9335,33,13.2


In [375]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_4/result_"+method+"/Bat.len10.MLResult.plk")
        # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.stringent.csv"
                                          , sep=',',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue
_df_to_report_200

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 12.00
Total result 6872
Total AMP from Tanspi: 250
-----------------RF-------------------------
no condi Accuracy Percentage = 6.00
Total result 6872
Total AMP from Tanspi: 250
-----------------XG-------------------------
no condi Accuracy Percentage = 10.80
Total result 6872
Total AMP from Tanspi: 250
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_4/result_CNN_RMSpop/Bat.len10.

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,1470,5402,30,12.0
RF,744,6128,15,6.0
XGBoost,1260,5612,27,10.8
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN
CNN_Adam,1066,5806,19,7.6
ProtCNN,1583,5289,37,14.8
BILSTM,1212,5660,25,10.0


# V5

# CD100

In [385]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}


_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_5/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)

        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

       # print("Total result " + str(len(Bat_ML)) )
       # print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
       # print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
       # print("Total match AMP " + str(len(AMP_match_Truth )))
       # print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
       # print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue
_df_to_report

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 86.42
no condi Accuracy Percentage = 74.76
100% Accuracy Percentage = 82.14
-----------------RF-------------------------
80% Accuracy Percentage = 87.56
no condi Accuracy Percentage = 71.39
100% Accuracy Percentage = 85.93
-----------------XG-------------------------
80% Accuracy Percentage = 87.64
no condi Accuracy Percentage = 74.88
100% Accuracy Percentage = 83.67
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
-----------------CNN_Adam-------------------------
80% Accuracy Percentage = 84.94
no condi Accuracy Percentage = 70.85
100% Accuracy Percentage = 83.36
-----------------ProtCNN-------------------------
80% Accuracy Percentage = 89.28
no condi Accuracy Percentage = 77.70
100% Accuracy Percentage = 87.22
-----------------BILSTM-------------------------


,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,118303,304493,5434,74.76,1343,82.14,4328,86.42
RF,154891,267905,5189,71.39,1405,85.93,4385,87.56
XGBoost,153558,269238,5443,74.88,1368,83.67,4389,87.64
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_Adam,124203,298593,5150,70.85,1363,83.36,4254,84.94
ProtCNN,177953,244843,5648,77.70,1426,87.22,4471,89.28
BILSTM,149414,273382,5567,76.59,1426,87.22,4430,88.46


In [386]:
for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/cd100/result_4/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output200/bat.len200.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)
        Bat_ML = Bat_ML[Bat_ML["length"]<= 200]
        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%")

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

        #print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        #print("Total match AMP " + str(len(AMP_match_Truth )))
        #print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report_200.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report_200.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report_200.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report_200.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report_200.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report_200.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report_200.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report_200.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except:
        continue
_df_to_report_200

-----------------Ensemble-------------------------
-----------------FastAI-------------------------
80% Accuracy Percentage = 81.20
no condi Accuracy Percentage = 72.37
100% Accuracy Percentage = 75.87
-----------------RF-------------------------
80% Accuracy Percentage = 84.40
no condi Accuracy Percentage = 72.12
100% Accuracy Percentage = 80.40
-----------------XG-------------------------
80% Accuracy Percentage = 85.55
no condi Accuracy Percentage = 75.57
100% Accuracy Percentage = 82.59
-----------------SVM-------------------------
-----------------ShortCNN-------------------------
-----------------CNN_RMSpop-------------------------
-----------------CNN_Adam-------------------------
80% Accuracy Percentage = 82.30
no condi Accuracy Percentage = 72.17
100% Accuracy Percentage = 79.69
-----------------ProtCNN-------------------------
80% Accuracy Percentage = 87.00
no condi Accuracy Percentage = 77.61
100% Accuracy Percentage = 83.51
-----------------BILSTM-------------------------


,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,85356,253882,4062,72.37,1072,75.87,3524,81.20
RF,100061,239177,4048,72.12,1136,80.40,3663,84.40
XGBoost,117465,221773,4242,75.57,1167,82.59,3713,85.55
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_Adam,98559,240679,4051,72.17,1126,79.69,3572,82.30
ProtCNN,140962,198276,4356,77.61,1180,83.51,3776,87.00
BILSTM,107339,231899,4210,75.00,1154,81.67,3644,83.96


# Transpi

In [384]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "AmPep30": "ShortCNN",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}
_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","AmPep30","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/transpi/result_5/result_"+method+"/Bat.len10.MLResult.plk")
           # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/BAT_truth_set.stringent.csv"
                                          , sep=',',header=0)
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")


        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(AMP_match_Truth ))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue
_df_to_report

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_5/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 12.00
Total result 11005
Total match AMP 30
Total AMP from Tanspi: 250
-----------------RF-------------------------
no condi Accuracy Percentage = 10.40
Total result 11005
Total match AMP 26
Total AMP from Tanspi: 250
-----------------XG-------------------------
no condi Accuracy Percentage = 15.60
Total result 11005
Total match AMP 39
Total AMP from Tanspi: 250
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_5/result_SVM/Bat.len10.MLResult.plk'
-----------------ShortCNN-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/transpi/result_5/result_ShortCNN/Bat.len10.MLResult.plk'
-----------------CNN_RMSpop-------------------------
[Errno 2] No such file or directory

,AMP,NonAMP,Match(Transpi),Accuracy %
Ensemble,NaN,NaN,NaN,NaN
FastAI,1660,9345,30,12.0
RF,1520,9485,26,10.4
XGBoost,1877,9128,39,15.6
SVM,NaN,NaN,NaN,NaN
AmPep30,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN
CNN_Adam,1741,9264,35,14.0
ProtCNN,2511,8494,49,19.6
BILSTM,2166,8839,47,18.8


# PWM

## CD100

In [23]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "CNN2D_Adam": "CNN2D_Adam",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}


_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(MMseq)","Accuracy %","_100_Match(MMseq)","_100_Accuracy %","_80_Match(MMseq)","_80_Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/pws/cd100/result/result_"+method+"/Bat.len10.MLResult.plk")
        Bat_len10_truth = pd.read_csv("/home/ubuntu/data/MMseqs.clus/output/BAT.alnRes.m8.len10.tsv"
                                          ,encoding = "ISO-8859-1", sep='\t',header=0)

        Bat_total_AMPS = len(Bat_len10_truth['ID_Query'].unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]

        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )

        # AMP only compares with filtering 80% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] >=0.8]
        filtered_BAT_total_AMPS_with80= len(filtered_BAT_len10_truth['ID_Query'].unique())
        AMP_match_Truth_80 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        accuracy_80 = accuracy_cal(len(AMP_match_Truth_80),filtered_BAT_total_AMPS_with80,"80%") 

        # AMP only compares with filtering 100% seq identity on ground truth
        filtered_BAT_len10_truth =Bat_len10_truth[Bat_len10_truth['sequence_identity'] == 1]
        filtered_BAT_total_AMPS_with100 = len(filtered_BAT_len10_truth['ID_Query'].unique())

        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = set(Bat_AMPs_ML["ID"]).intersection(set(Bat_len10_truth['ID_Query']))
        accuracy = accuracy_cal(len(AMP_match_Truth),Bat_total_AMPS,"no condi")

        AMP_match_Truth_100 = set(Bat_AMPs_ML["ID"]).intersection(set(filtered_BAT_len10_truth['ID_Query']))
        # 100%
        accuracy_100 = accuracy_cal(len(AMP_match_Truth_100),filtered_BAT_total_AMPS_with100,"100%")

       # print("Total result " + str(len(Bat_ML)) )
       # print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
       # print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
       # print("Total match AMP " + str(len(AMP_match_Truth )))
       # print("Total AMP  from MMSEQ: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
       # print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(MMseq)']=len(AMP_match_Truth )
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        _df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        _df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        _df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        _df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue
_df_to_report

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/pws/cd100/result/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
80% Accuracy Percentage = 83.37
no condi Accuracy Percentage = 71.36
100% Accuracy Percentage = 78.47
-----------------RF-------------------------
80% Accuracy Percentage = 77.46
no condi Accuracy Percentage = 62.06
100% Accuracy Percentage = 75.96
-----------------XG-------------------------
80% Accuracy Percentage = 72.42
no condi Accuracy Percentage = 60.24
100% Accuracy Percentage = 74.13
-----------------SVM-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/pws/cd100/result/result_SVM/Bat.len10.MLResult.plk'
-----------------CNN2D_Adam-------------------------
80% Accuracy Percentage = 79.59
no condi Accuracy Percentage = 64.92
100% Accuracy Percentage = 75.78
-----------------CNN_RMSpop-------------------------
[Errno 2] No such file or directory: '/

,AMP,NonAMP,Match(MMseq),Accuracy %,_100_Match(MMseq),_100_Accuracy %,_80_Match(MMseq),_80_Accuracy %
Ensemble,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FastAI,70804,351992,5187,71.36,1283,78.47,4175,83.37
RF,90669,332127,4511,62.06,1242,75.96,3879,77.46
XGBoost,122506,300290,4379,60.24,1212,74.13,3627,72.42
SVM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN2D_Adam,84824,337972,4719,64.92,1239,75.78,3986,79.59
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNN_Adam,90622,332174,4732,65.1,1265,77.37,4035,80.57
ProtCNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BILSTM,112299,310497,4682,64.41,1267,77.49,3919,78.25


## Transpi

In [21]:
method_dict = {
    'Ensemble': "Ensemble",
    'FastAI': 'FastAI', 
    'RF' : "RF",
    "XGBoost": "XG",
    "SVM": "SVM",
    "CNN2D_Adam": "CNN2D_Adam",
    "CNN_RMSpop": "CNN_RMSpop",
    "CNN_Adam": "CNN_Adam",
    "ProtCNN" : "ProtCNN",
    "BILSTM" : "BILSTM"
}
_df_to_report_200 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report_30 =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])
_df_to_report =  pd.DataFrame( columns=["AMP","NonAMP","Match(Transpi)","Accuracy %"],
    index = ['Ensemble','FastAI', 'RF',"XGBoost","SVM","CNN2D_Adam","CNN_RMSpop","CNN_Adam","ProtCNN","BILSTM"])

with open('TransPi/consensus/bat.final.truth') as f:
    bat_lines = [line.rstrip() for line in f]

for key, value in method_dict.items():
    try:
        method = value
        print("-----------------"+ method +"-------------------------")
        Bat_ML =  pd.read_pickle("/mnt/vdb/Bat/pws/transpi/result/result_2/result_"+method+"/Bat.len10.MLResult.plk")
           # BAT_truth_set.stringent.csv , BAT_truth_set.csv
        Bat_len10_truth = pd.read_csv("TransPi/truth_no_ambiguous//BAT_truth_set.stringent.csv"
                                          , sep=',',header=0)
        Bat_total_AMPS = len(Bat_len10_truth.prot_id.unique())
        Bat_AMPs_ML = Bat_ML[Bat_ML["class"] == 0]


        # AMP and Non AMP compare with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_ML,
        #                Bat_len10_truth
        #               )
        # AMP only compares with ground truth
        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                Bat_AMPs_ML,
        #                Bat_len10_truth
        #               )


        #plot_ven_AMP_NonAMP("Bat_"+key+"_AMP","Bat_MMSEQ_GroundTruth",
        #                    Bat_AMPs_ML,
        #                    filtered_BAT_len10_truth
        #               )


        # AMP match Truthset
        AMP_match_Truth = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(Bat_len10_truth.prot_id.to_list()))])
        accuracy= accuracy_cal(AMP_match_Truth,Bat_total_AMPS,"no condi")
        # AMP match consensus
        AMP_match_Truth_consensus = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.isin(bat_lines)])
        accuracy_2M= accuracy_cal(AMP_match_Truth_consensus,len(bat_lines),"no condi")

        print("Total result " + str(len(Bat_ML)) )
        #print("Total predicted AMP " + str(len(Bat_AMPs_ML) ) )
        #print("Total predicted NonAMP " + str(len(Bat_ML) -len(Bat_AMPs_ML)) )
        print("Total match AMP " + str(AMP_match_Truth ))
        print("Total AMP from Tanspi: " + str( Bat_total_AMPS) )
        #print("Total match AMP with_100 " + str( len(AMP_match_Truth_100) ))
        #print("Total AMP with_100 from MMSEQ: " + str( filtered_BAT_total_AMPS_with100) )


        _df_to_report.loc[key, 'AMP']=len(Bat_AMPs_ML) 
        _df_to_report.loc[key, 'NonAMP']=len(Bat_ML) -len(Bat_AMPs_ML)
        _df_to_report.loc[key, 'Match(Transpi)']=AMP_match_Truth 
        _df_to_report.loc[key, 'Accuracy_2M %']= round(accuracy_2M, 2)
        _df_to_report.loc[key, 'Accuracy %']= round(accuracy, 2)
        #_df_to_report.loc[key, '_100_Match(MMseq)']= int(len(AMP_match_Truth_100))
        #_df_to_report.loc[key, '_100_Accuracy %']=round(accuracy_100, 2)
        #_df_to_report.loc[key, '_80_Match(MMseq)']= int(len(AMP_match_Truth_80))
        #_df_to_report.loc[key, '_80_Accuracy %']=round(accuracy_80, 2)

        #print("-------------------------------------------------")
    except Exception as e:
        print(e)
        continue
_df_to_report

-----------------Ensemble-------------------------
[Errno 2] No such file or directory: '/mnt/vdb/Bat/pws/transpi/result/result_2/result_Ensemble/Bat.len10.MLResult.plk'
-----------------FastAI-------------------------
no condi Accuracy Percentage = 43.81
no condi Accuracy Percentage = 23.08
Total result 10180
Total match AMP 99
Total AMP from Tanspi: 226
-----------------RF-------------------------
no condi Accuracy Percentage = 19.47
no condi Accuracy Percentage = 7.69
Total result 10180
Total match AMP 44
Total AMP from Tanspi: 226
-----------------XG-------------------------
no condi Accuracy Percentage = 28.76
no condi Accuracy Percentage = 23.08
Total result 10180
Total match AMP 65
Total AMP from Tanspi: 226
-----------------SVM-------------------------
no condi Accuracy Percentage = 11.06
no condi Accuracy Percentage = 15.38
Total result 10180
Total match AMP 25
Total AMP from Tanspi: 226
-----------------CNN2D_Adam-------------------------
[Errno 2] No such file or directory: 

,AMP,NonAMP,Match(Transpi),Accuracy %,Accuracy_2M %
Ensemble,NaN,NaN,NaN,NaN,NaN
FastAI,3816,6364,99,43.81,23.08
RF,1735,8445,44,19.47,7.69
XGBoost,2375,7805,65,28.76,23.08
SVM,1266,8914,25,11.06,15.38
CNN2D_Adam,NaN,NaN,NaN,NaN,NaN
CNN_RMSpop,NaN,NaN,NaN,NaN,NaN
CNN_Adam,NaN,NaN,NaN,NaN,NaN
ProtCNN,NaN,NaN,NaN,NaN,NaN
BILSTM,NaN,NaN,NaN,NaN,NaN


In [10]:
# Consensus

with open('TransPi/consensus/bat.final.truth') as f:
    bat_lines = [line.rstrip() for line in f]
print(bat_lines)

['SOAP.k41.scaffold3204.p1', 'SOAP.k61.scaffold787.p1', 'SPADES.k25.NODE_1519_length_1362_cov_42.465221_g1145_i0.p1', 'SPADES.k53.NODE_1887_length_1029_cov_50.609631_g1602_i0.p1', 'SPADES.k53.NODE_940_length_1415_cov_58.640969_g785_i0.p1', 'TRINITY_DN1969_c0_g1_i1.p1', 'TRINITY_DN378_C0_G1_I6.P1', 'TRINITY_DN4361_c0_g2_i3.p1', 'TRINITY_DN7196_c0_g1_i6.p1', 'TRINITY_DN7253_c0_g1_i1.p1', 'TRINITY_DN8574_c0_g2_i1.p1', 'TRINITY_DN951_c0_g1_i5.p1', 'TRANSABYSS.K61.R47039.P1']


In [7]:
len(lines)

13

In [14]:
AMP_match_Truth_consensus = len(Bat_AMPs_ML[Bat_AMPs_ML.ID.str.contains('|'.join(bat_lines))])

In [15]:
accuracy= accuracy_cal(AMP_match_Truth_consensus,len(bat_lines),"no condi")

no condi Accuracy Percentage = 15.38


In [18]:
'|'.join(bat_lines)

'SOAP.k41.scaffold3204.p1|SOAP.k61.scaffold787.p1|SPADES.k25.NODE_1519_length_1362_cov_42.465221_g1145_i0.p1|SPADES.k53.NODE_1887_length_1029_cov_50.609631_g1602_i0.p1|SPADES.k53.NODE_940_length_1415_cov_58.640969_g785_i0.p1|TRINITY_DN1969_c0_g1_i1.p1|TRINITY_DN378_C0_G1_I6.P1|TRINITY_DN4361_c0_g2_i3.p1|TRINITY_DN7196_c0_g1_i6.p1|TRINITY_DN7253_c0_g1_i1.p1|TRINITY_DN8574_c0_g2_i1.p1|TRINITY_DN951_c0_g1_i5.p1|TRANSABYSS.K61.R47039.P1'